In [1]:
import math
import numpy as np
import pandas as pd
from openpyxl import load_workbook

# ----- CONFIG -----
FILES = [
    "SBA_compos_analysis.xlsx",
    "Thermo_compos_analysis.xlsx",
    "Acme_compos_analysis.xlsx",
    "Kaun_compos_analysis.xlsx",
    "Ignitis_compos_analysis.xlsx",
]
SHEET = "Raw Data"

# Column names exactly as they appear in your sheet
COL_QUALITY = "Quality"
COL_RANK = "Rank"
COL_LOG_RANK = "Log_Rank"
COL_N = "query_occurrences"
COL_QS = "Quality_Score"
COL_BMQ = "BMQ"

def log_scale_with_clipping_lower_better(value, lo, hi):
    v = np.clip(value, lo, hi)
    return 1.0 - (np.log(v) - np.log(lo)) / (np.log(hi) - np.log(lo))

def log_scale_with_clipping_higher_better(value, lo, hi):
    v = np.clip(value, lo, hi)
    return (np.log(v) - np.log(lo)) / (np.log(hi) - np.log(lo))

def linear_scale_with_clipping(value, lo, hi):
    v = np.clip(value, lo, hi)
    return (v - lo) / (hi - lo)

def compute_bmq(df):
    """Return a pandas Series with BMQ following your Option 2 spec."""
    # Basic validation
    required = {COL_QUALITY, COL_RANK, COL_LOG_RANK, COL_N, COL_QS}
    missing = [c for c in required if c not in df.columns]
    if missing:
        raise ValueError(f"Missing required columns: {missing}")

    df = df.copy()

    # R_input = Log_Rank * 100, and +1 for the first data row (if any)
    R_input = df[COL_LOG_RANK].astype(float) * 100.0
    if len(R_input) > 0:
        R_input.iloc[0] = R_input.iloc[0] + 1.0

    # Scaled components
    # P (Rank): lower is better, log scale, range 1..1e8
    scaled_P = log_scale_with_clipping_lower_better(df[COL_RANK].astype(float), 1.0, 1e8)

    # R (R_input): lower is better, log scale, range 1..100
    scaled_R = log_scale_with_clipping_lower_better(R_input, 1.0, 100.0)

    # n (query_occurrences): higher is better, log scale, range 1..30
    scaled_n = log_scale_with_clipping_higher_better(df[COL_N].astype(float), 1.0, 30.0)

    # Q (Quality_Score): linear, range 1..3
    scaled_Q = linear_scale_with_clipping(df[COL_QS].astype(float), 1.0, 3.0)

    score = 0.25 * (scaled_P + scaled_R + scaled_n + scaled_Q)

    # Only for Quality == 'A'; others blank/NaN
    bmq = np.where(df[COL_QUALITY].astype(str).str.upper() == "A", score, np.nan)

    return pd.Series(bmq, index=df.index)

def write_bmq_to_sheet(xlsx_path, sheet_name=SHEET):
    # Read with pandas
    df = pd.read_excel(xlsx_path, sheet_name=sheet_name)

    # Compute BMQ
    bmq_series = compute_bmq(df)

    # Open with openpyxl to write only the BMQ column (preserve everything else)
    wb = load_workbook(filename=xlsx_path)
    if sheet_name not in wb.sheetnames:
        wb.close()
        raise ValueError(f"Sheet '{sheet_name}' not found in '{xlsx_path}'.")

    ws = wb[sheet_name]

    # Determine header row (assume row 1)
    headers = [cell.value for cell in ws[1]]

    # Find or append BMQ column
    try:
        bmq_col_idx = headers.index(COL_BMQ) + 1  # 1-based
    except ValueError:
        bmq_col_idx = ws.max_column + 1
        ws.cell(row=1, column=bmq_col_idx, value=COL_BMQ)

    # Write values (row 2 onward)
    for i, val in enumerate(bmq_series, start=2):
        # Write None if NaN to keep the cell blank
        out_val = None if (pd.isna(val)) else float(val)
        ws.cell(row=i, column=bmq_col_idx, value=out_val)

    wb.save(xlsx_path)
    wb.close()

for path in FILES:
    print(f"Processing: {path}")
    try:
        write_bmq_to_sheet(path, SHEET)
        print(f"  ✓ BMQ written to '{SHEET}' in {path}")
    except Exception as e:
        print(f"  ✗ Failed on {path}: {e}")


Processing: SBA_compos_analysis.xlsx
  ✓ BMQ written to 'Raw Data' in SBA_compos_analysis.xlsx
Processing: Thermo_compos_analysis.xlsx
  ✓ BMQ written to 'Raw Data' in Thermo_compos_analysis.xlsx
Processing: Acme_compos_analysis.xlsx
  ✓ BMQ written to 'Raw Data' in Acme_compos_analysis.xlsx
Processing: Kaun_compos_analysis.xlsx
  ✓ BMQ written to 'Raw Data' in Kaun_compos_analysis.xlsx
Processing: Ignitis_compos_analysis.xlsx
  ✓ BMQ written to 'Raw Data' in Ignitis_compos_analysis.xlsx


In [ ]:
import pandas as pd

# ----- CONFIG -----
FILES = [
    "SBA_compos_analysis.xlsx",
    "Thermo_compos_analysis.xlsx",
    "Acme_compos_analysis.xlsx",
    "Kaun_compos_analysis.xlsx",
    "Ignitis_compos_analysis.xlsx",
]
SHEET = "Raw Data"
OUTPUT = "full_pr.xlsx"

def extract_company_name(filename: str) -> str:
    # Take the part before the first underscore
    return filename.split("_", 1)[0]

def merge_raw_data(files, sheet_name, output_path):
    all_frames = []
    for path in files:
        try:
            df = pd.read_excel(path, sheet_name=sheet_name)
        except Exception as e:
            print(f"✗ Failed reading {path}: {e}")
            continue
        company = extract_company_name(path)
        df.insert(0, "company", company)  # prepend column
        all_frames.append(df)

    if not all_frames:
        raise RuntimeError("No data was read from any files.")

    merged = pd.concat(all_frames, ignore_index=True)
    merged.to_excel(output_path, sheet_name="full", index=False)
    print(f"✓ Merged {len(all_frames)} sheets into {output_path} ({len(merged)} rows).")


merge_raw_data(FILES, SHEET, OUTPUT)
